<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1ea1aed1e27aaacbcd726663028004c8144cf0c028c0eae5688f53ef4f7e2452
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 15:20:37
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: -45.82 K (-0.31%)
Current PnL: -20.24 L (-13.26%)
CY Booked + Current PnL: -6.72 L (-4.4%)
-------------------
Total profit:  1.89 L
Total loss:  -22.13 L
-------------------
Total Booked + Current PnL: 20.26 L (16.13%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 89.54 L (61.67%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,100.85,50.0,M-SC,2.68,86578.0,-14199.0,14294.0,-0.47,-14.09,16.51,0.09,245.0,-0.99,0.60,13.74,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.00,141.63,83.0,M-SC,3.18,158584.0,-15975.0,16001.0,3.76,-9.15,10.09,0.02,221.0,-1.00,1.10,37.94,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.34,88770.0,-1908.0,16813.0,0.07,-2.10,18.94,16.44,101.0,-0.11,0.62,17.65,X40,ATH,PHARMA
34,ICICIGI,2252.93,-12.45,59.0,X-MC,7.57,189060.0,14740.0,20456.0,0.21,8.46,10.82,20.19,91.0,0.72,1.32,24.85,X40,ATH,INSURANCE
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198387.0,-1751.0,22636.0,-0.32,-0.87,11.41,10.44,4.0,-0.08,1.38,4.64,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,-3.39,81.0,X-LC,7.05,243742.0,-8026.0,61228.0,4.24,-3.19,25.12,21.13,27.0,-0.13,1.70,31.11,X40,ATH,PAINTS
38,INDIGOPNTS,1408.00,141.63,83.0,M-SC,3.18,158584.0,-15975.0,16001.0,3.76,-9.15,10.09,0.02,221.0,-1.00,1.10,37.94,OX40N,NTT,PAINTS
13,BERGEPAINT,680.00,-12.97,70.0,X-MC,1.62,233867.0,6528.0,41535.0,3.10,2.87,17.76,21.14,106.0,0.16,1.63,31.82,XY24,NTT,PAINTS
55,RAJESHEXPO,518.00,1692.93,55.0,L-SC,3.49,48908.0,-88269.0,88362.0,2.26,-64.35,180.67,0.07,267.0,-1.00,0.34,21.64,OX40N,NTT,JEWELLERY
46,KANSAINER,340.00,-67.42,52.0,H-SC,2.41,224478.0,-45189.0,81530.0,1.58,-16.76,36.32,13.47,138.0,-0.55,1.56,14.14,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-55.29,51.0,H-SC,9.44,256470.0,-30035.0,287375.0,-5.59,-10.48,112.05,89.82,134.0,-0.10,1.79,35.69,XY24,NTT,FINANCE
47,KPIGREEN,731.05,-0.19,38.0,H-SC,10.17,118957.0,-6340.0,65272.0,-2.85,-5.06,54.87,47.03,141.0,-0.10,0.83,48.94,MH,ATH,POWER
7,ASIANTILES,137.00,7483.33,61.0,L-SC,15.78,83224.0,-10586.0,87202.0,-2.55,-11.28,104.78,81.67,269.0,-0.12,0.58,61.17,XR,NTT,CERAMICS
54,QUESS,986.00,-52.62,29.0,X-SC,37.29,46997.0,-18009.0,168935.0,-2.19,-27.70,359.46,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
69,SURYODAY,240.00,51.32,40.0,H-SC,13.53,138516.0,-40555.0,106283.0,-1.97,-22.65,76.73,36.71,135.0,-0.38,0.96,37.44,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152160.0,1215.0,108247.0,0.08,0.80,71.14,72.51,53.0,0.01,1.06,6.88,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,37.0,H-MC,5.89,208801.0,-773.0,75440.0,-0.69,-0.37,36.13,35.63,NaN,-0.01,1.45,16.24,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,141.63,83.0,M-SC,3.18,158584.0,-15975.0,16001.0,3.76,-9.15,10.09,0.02,221.0,-1.00,1.10,37.94,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,100.85,50.0,M-SC,2.68,86578.0,-14199.0,14294.0,-0.47,-14.09,16.51,0.09,245.0,-0.99,0.60,13.74,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,52.0,H-SC,2.41,224478.0,-45189.0,81530.0,1.58,-16.76,36.32,13.47,138.0,-0.55,1.56,14.14,XY24,NTT,PAINTS
66,SIS,528.0,2021.24,47.0,H-SC,2.14,85604.0,-25428.0,49034.0,-0.36,-22.90,57.28,21.26,156.0,-0.52,0.60,15.66,OX40N,NTT,MISC
43,JCHAC,2282.0,19965.22,64.0,M-SC,1.48,98017.0,-29788.0,29778.0,0.05,-23.31,30.38,-0.01,232.0,-1.00,0.68,10.99,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152160.0,1215.0,108247.0,0.08,0.80,71.14,72.51,53.0,0.01,1.06,6.88,XR,NTT,IT
37,INDIAMART,4810.62,-52.96,56.0,H-SC,4.76,131249.0,7913.0,123715.0,-1.12,6.42,94.26,106.72,119.0,0.06,0.91,31.29,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,55.0,M-SC,36.10,160380.0,11160.0,133019.0,0.70,7.48,82.94,96.62,208.0,0.08,1.12,73.57,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.96,56.0,H-SC,4.76,131249.0,7913.0,123715.0,-1.12,6.42,94.26,106.72,119.0,0.06,0.91,31.29,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152160.0,1215.0,108247.0,0.08,0.80,71.14,72.51,53.0,0.01,1.06,6.88,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,37.0,H-MC,5.89,208801.0,-773.0,75440.0,-0.69,-0.37,36.13,35.63,NaN,-0.01,1.45,16.24,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.71,36.0,M-SC,9.17,112599.0,-2199.0,96238.0,-0.24,-1.92,85.47,81.92,223.0,-0.02,0.78,33.30,XR,NTT,DURABLES
47,KPIGREEN,731.05,-0.19,38.0,H-SC,10.17,118957.0,-6340.0,65272.0,-2.85,-5.06,54.87,47.03,141.0,-0.10,0.83,48.94,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235824.0,-25795.0,145008.0,-0.22,-9.86,61.49,45.57,7.0,-0.18,1.64,0.35,AR,NTT,ELECTRICAL
26,GLAXO,3466.20,-21.19,29.0,X-MC,5.35,199344.0,60.0,77944.0,-0.59,0.03,39.10,39.15,96.0,0.00,1.39,29.04,X40,ATH,PHARMA
25,GILLETTE,11206.78,-17.30,29.0,X-MC,5.77,254175.0,279.0,82022.0,-0.43,0.11,32.27,32.42,105.0,0.00,1.77,15.23,X40,ATH,FMCG
54,QUESS,986.00,-52.62,29.0,X-SC,37.29,46997.0,-18009.0,168935.0,-2.19,-27.70,359.46,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
53,PGHH,17907.65,-33.19,30.0,X-MC,1.55,193500.0,-7320.0,75117.0,0.23,-3.65,38.82,33.76,80.0,-0.10,1.35,1.36,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198387.0,-1751.0,22636.0,-0.32,-0.87,11.41,10.44,4.0,-0.08,1.38,4.64,X40,NTT,FMCG
53,PGHH,17907.65,-33.19,30.0,X-MC,1.55,193500.0,-7320.0,75117.0,0.23,-3.65,38.82,33.76,80.0,-0.10,1.35,1.36,X40,ATH,FMCG
65,SIEMENS,4671.50,-4.54,44.0,H-LC,1.66,153345.0,-32750.0,80230.0,0.34,-17.60,52.32,25.51,15.0,-0.41,1.07,13.00,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-7.74,44.0,X-MC,3.22,307776.0,-20215.0,126742.0,0.52,-6.16,41.18,32.48,92.0,-0.16,2.14,4.56,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-6.81,59.0,X-LC,3.30,244343.0,10199.0,38435.0,-1.04,4.36,15.73,20.77,86.0,0.27,1.70,11.58,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-42.01,30.0,X-SC,8.95,86768.0,-41902.0,87202.0,-1.14,-32.57,100.50,35.20,219.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-52.62,29.0,X-SC,37.29,46997.0,-18009.0,168935.0,-2.19,-27.70,359.46,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235824.0,-25795.0,145008.0,-0.22,-9.86,61.49,45.57,7.0,-0.18,1.64,0.35,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-33.19,30.0,X-MC,1.55,193500.0,-7320.0,75117.0,0.23,-3.65,38.82,33.76,80.0,-0.10,1.35,1.36,X40,ATH,FMCG
10,BAJAJHFL,181.50,-16.17,37.0,X-MC,6.43,176464.0,-24042.0,121195.0,-0.40,-11.99,68.68,48.45,90.0,-0.20,1.23,1.76,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.62,29.0,X-SC,37.29,46997.0,-18009.0,168935.0,-2.19,-27.70,359.46,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.75,42.0,X-SC,3.58,80831.0,-64329.0,140258.0,-1.07,-44.32,173.52,52.31,136.0,-0.46,0.56,8.20,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-42.01,30.0,X-SC,8.95,86768.0,-41902.0,87202.0,-1.14,-32.57,100.50,35.20,219.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.05,48.0,X-MC,3.34,88770.0,-1908.0,16813.0,0.07,-2.10,18.94,16.44,101.0,-0.11,0.62,17.65,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.48,35.0,X-SC,0.97,106425.0,-21513.0,68644.0,-0.06,-16.82,64.50,36.84,143.0,-0.31,0.74,8.95,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.76,60.0,X-LC,8.78,292020.0,-53936.0,120633.0,-0.80,-15.59,41.31,19.28,1.0,-0.45,2.03,7.96,X40,ATH,IT
40,INFY,2275.00,-15.65,63.0,X-LC,4.33,327935.0,15047.0,156655.0,-0.78,4.81,47.77,54.87,3.0,0.10,2.28,13.49,X40,BTT,IT
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198387.0,-1751.0,22636.0,-0.32,-0.87,11.41,10.44,4.0,-0.08,1.38,4.64,X40,NTT,FMCG
81,VBL,671.64,-19.89,43.0,X-LC,5.88,288408.0,-27434.0,140109.0,-1.58,-8.69,48.58,35.67,5.0,-0.20,2.01,3.96,X40N,ATH,FMCG
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235824.0,-25795.0,145008.0,-0.22,-9.86,61.49,45.57,7.0,-0.18,1.64,0.35,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,301.26,78.0,L-MC,6.71,49254.0,-27952.0,53347.0,-0.03,-36.20,108.31,32.89,258.0,-0.52,0.34,35.64,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,45.0,L-SC,25.84,77568.0,-35981.0,76032.0,-0.12,-31.69,98.02,35.27,268.0,-0.47,0.54,89.85,XR,NTT,HOTELS
7,ASIANTILES,137.00,7483.33,61.0,L-SC,15.78,83224.0,-10586.0,87202.0,-2.55,-11.28,104.78,81.67,269.0,-0.12,0.58,61.17,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,49.0,H-SC,7.51,91635.0,-6345.0,27949.0,-0.83,-6.48,30.50,22.05,152.0,-0.23,0.64,33.20,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.96,56.0,H-SC,4.76,131249.0,7913.0,123715.0,-1.12,6.42,94.26,106.72,119.0,0.06,0.91,31.29,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.00,141.63,83.0,M-SC,3.18,158584.0,-15975.0,16001.0,3.76,-9.15,10.09,0.02,221.0,-1.00,1.10,37.94,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-3.39,81.0,X-LC,7.05,243742.0,-8026.0,61228.0,4.24,-3.19,25.12,21.13,27.0,-0.13,1.70,31.11,X40,ATH,PAINTS
79,VAIBHAVGBL,521.00,73.00,62.0,H-SC,7.56,149996.0,-32779.0,143846.0,0.40,-17.93,95.90,60.77,125.0,-0.23,1.04,35.08,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7483.33,61.0,L-SC,15.78,83224.0,-10586.0,87202.0,-2.55,-11.28,104.78,81.67,269.0,-0.12,0.58,61.17,XR,NTT,CERAMICS
5,ANGELONE,3033.00,25.51,73.0,X-SC,7.07,222548.0,31542.0,26149.0,1.29,16.51,11.75,30.21,157.0,1.21,1.55,40.84,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.83
2,50,76.43


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.30
LC    27.84
MC    27.06
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.73
X40      20.75
X40N     12.33
XY25     10.15
XR        9.68
AR        9.10
OX40N     6.58
X200      1.83
SR        1.03
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.75
X-MC    22.96
X-LC    19.88
M-SC    12.50
X-SC     5.59
H-LC     4.54
H-MC     3.76
M-LC     2.43
L-SC     1.46
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.77,-4.50,38.29
IT,12.72,-14.97,76.51
FINANCE,12.01,-9.69,59.42
MISC,7.66,-18.18,77.80
PAINTS,5.99,-7.28,23.27
ELECTRICAL,5.59,-12.48,52.36
INSURANCE,4.51,2.20,32.26
AUTO,3.53,-13.37,122.06
PHARMA,3.46,-0.53,34.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3323609.0,20
AR,1266608.0,10
XR,1263464.0,13
X40,989148.0,14
X40N,696106.0,9
OX40N,539309.0,9
XY25,477579.0,7
SR,271863.0,2
MH,65272.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3420021.0,24
X-MC,1388873.0,16
M-SC,1249346.0,15
X-LC,798279.0,11
X-SC,544785.0,6
H-MC,330972.0,3
H-LC,286075.0,3
L-SC,251596.0,3
M-LC,156043.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1183942.0      6
           AR         841499.0      5
           XR         794751.0      7
M-SC       XY24       733072.0      6
X-MC       XY24       552647.0      4
           X40        490083.0      8
X-LC       X40        411863.0      5
X-SC       X40N       288648.0      4
H-SC       SR         271863.0      2
           OX40N      262694.0      3
X-LC       X40N       245491.0      3
H-LC       AR         225238.0      2
X-MC       XY25       184176.0      2
H-MC       XY24       182218.0      1
X-SC       XY24       168935.0      1
L-SC       XR         163234.0      2
X-MC       X40N       161967.0      2
M-SC       XR         143885.0      2
           XY25       133019.0      1
           AR         124431.0      2
           OX40N      114939.0      4
M-LC       XR         108247.0      1
L-SC       OX40N       88362.0      1
X-SC       X40         87202.0      1
H-MC       AR          75440.0      1
X-LC       XY25        75043.0      2
H-MC       OX40N       73314.0      1
X-LC       XY24        65882.0      1
H-SC       MH          65272.0      1
H-LC       X200        60837.0      1
L-MC       XR          53347.0      1
M-LC       XY25        47796.0      1
L-LC       XY25        37545.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 38.0 seconds
